In [99]:
import requests
import time

from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#from selenium.webdriver.chrome.options import Options
#from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome()

class scraper:

    def getURL(url):
        driver.get(url) 

    def getTitle():
        print(driver.title)

    def closeSession():
        time.sleep(3)
        driver.quit()

    def getSourceCode():
        print(driver.page_source)
    
    def search(search_term):
        try:
            button = driver.find_element(By.ID, 'nav-searchbar-btn')
            button.click()
            try:
                search_bar = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "sb")))
                try:
                    search_bar.send_keys(search_term)
                    search_bar.send_keys(Keys.RETURN) # Return = Enter
                except:
                    print("Exception: No search term input")
            except:
                print("Exception: No search bar found")
        except:
            print("Exception: No search button found")

    def home():
        title = driver.find_element(By.ID, 'nav-image')
        title.click()

    def findRecipeList():
        button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.LINK_TEXT, 'Recipes')))
        button.click()

    def acceptCookies():
        try:
            cookie_button = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]')
            cookie_button.click()
            print("Removed Cookies")
        except:
            print("Exeption: Didnt Find Cookie Button")
    
    def getRecipes():
        main = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'index-item-container')))      
        print("Found results")
        articles = main.find_elements(By.TAG_NAME, 'li')
        print("Number of Recipes:", len(articles))

        for i in articles:
            print("Recipe:" , i.text)


scraper.getURL('https://www.pickuplimes.com/')
scraper.getTitle()
#scraper.getSourceCode()
scraper.acceptCookies()
#scraper.search("lemon")
#time.sleep(3)
scraper.home()
scraper.findRecipeList()
scraper.getRecipes()
#time.sleep(3)
scraper.closeSession()

# IDEALS
# accept cookies
# get search results
# get search results on more than the first page
# bypass login
# a method that makes a list containing all buttons and cycles through them looking for a link_text given as an argument

Pick Up Limes
Removed Cookies
Found results
Number of Recipes: 12
Recipe: High Protein Vegan Pancakes
30 min
Breakfast
Recipe: Leek and Spinach Tofu Quiche
1 hr + 7 min
Meal
Breakfast
Recipe: Avocado Toast with Chipotle Bean Spread
17 min
Meal
Recipe: Vegan Chocolate Cake
1 hr + 20 min
Dessert
Recipe: Vegan Chocolate Buttercream Frosting
8 min
Dessert
Recipe: Homemade Beet Pasta
1 hr + 15 min
Meal
Make-ahead
10 ingredients or less
Freezer-friendly
Recipe: Creamy Vegan Harissa Pasta
15 min
Meal
Recipe: Miso Sesame Soba Noodles
20 min
Meal
Recipe: Mushroom & Kidney Bean Coconut Curry
20 min
Meal
Freezer-friendly
Recipe: Vegan Onigirazu: Rice Sandwiches
43 min
Meal
Make-ahead
10 ingredients or less
On-the-go
Recipe: Chickpea Caesar Salad Wrap
19 min
Meal
10 ingredients or less
No-bake
On-the-go
Recipe: Dill Pasta Salad with a Tahini Dressing
20 min
Meal
Side
Salad
End


In [85]:
# PROJECT: GET SEARCH RESULTS
def getRecipes():
    main = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'index-item-container')))      
    print("Found results")
    articles = main.find_elements(By.CLASS_NAME, 'flex-item slide-up-ani')
    print("Found recipies")

    for article in articles:
        print("hi")
        title = articles.find_elements(By.NAME, 'h3')
        print(title.text)
        print("something")

In [130]:
# ZOOPLA
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

driver = webdriver.Chrome()  
url = "https://zoopla.co.uk" 
driver.get(url)

time.sleep(5) # Slow it down otherwise it will miss the button 

cookie_button = "Accept all cookies"
driver.find_element(By.LINK_TEXT, cookie_button)

try: 
    frame_id = "gdpr-consent-notice"
    driver.switch_to.frame(frame_id)
    print("Switched to frame")

    #accept_cookies_button = driver.find_element(By.XPATH, '/*[@id="save"]/span[1]/div/span') # XPath for button
    #print("Found Button")
except:
    print("No Cookie Button")
    pass

time.sleep(3)
driver.quit()
#try:
    #accept_cookies_button = driver.find_element(By.Tag_Name, "button") # XPath for button
    #print("Find Button")
    #accept_cookies_button.click() # Click the button 
    # If there isn't a cookie button 
#except: 
    #print("No Cookies Detected")
    #pass

gdpr-consent-notice
Switched to frame


In [20]:
my_list = [1, 2, 3]
print(my_list * 2)


[1, 2, 3, 1, 2, 3]
